# Aggregating GH Archive Data with jq

- toc: false
- badges: false
- comments: true
- sticky_rank: 1
- author: "<a href='https://twitter.com/rajkstats'>Raj Kumar</a>"
- description: "Tutorial on how to aggregate json data with jq"
- image: /images/copied_from_nb/agg-data-with-jq/blog-head.png
- categories: [jq, bash]

<img src="agg-data-with-jq/blog-head.png" width="700" height="400"/>

## Motivation
Transforming the raw data into a shape that's ready for analytics

## Goal
In this tutorial, we will be working with **GH Archive data** which is in `JSON` format, so before we move forward, we will provide some background on the source and dataset in the next section taken. End goal of this tutorial is to be able to write a bash script which should be able to aggregate the github raw logs using the fields of interest. 

## What is jq?
 - [jq](https://stedolan.github.io/jq/) is a lightweight and flexible command-line JSON processor.

- jq is like `sed` for JSON data - you can use it to slice and filter and map and transform structured data with the same ease that `sed`, `awk`, `grep` that lets you play with text.

- jq can mangle the data format that you have into the one that you want with very little effort, and the program to do so is often shorter and simpler than you'd expect

This command line processor becomes very handy for folks who works with json data. 

## Installation
jq is written in C and has no runtime dependencies, so it should be possible to build it for nearly any platform. Prebuilt binaries are available for Linux, OS X and Windows.

[Download here](https://stedolan.github.io/jq/download/)

**Credits**:[Stephen Dolan](https://stedolan.github.io/jq/)

To confirm the installation, type the following in terminal:

In [19]:
!jq --version

jq-1.6


>Important: Before we move forward, I would like to highlight here that anywhere you see '!','%' in the code has been done only for notebooks, remove it if you're not working with notebooks

## Download Dataset

**Data Source**: [GH Archive](https://www.gharchive.org/)

- GH Archive is an open source project to record the public GitHub timeline, archive it, and make it easily accessible for further analysis
- GitHub provides 20+ event types, which range from new commits and fork events, to opening new tickets, commenting, and adding members to a project. These events are aggregated into hourly archives
- Each archive contains JSON encoded events as reported by the GitHub API. You can download the raw data and apply own processing to it - e.g. write a custom aggregation script, import it into a database, and so on!

We will download one hour data, simply go to your terminal and follow the following steps 

In [13]:
%cd /Users/raj/Desktop/agg-data-with-jq/00_data/ 
! wget https://data.gharchive.org/2021-09-01-15.json.gz

/Users/raj/Desktop/agg-data-with-jq/00_data
--2021-09-11 16:30:29--  https://data.gharchive.org/2021-09-01-15.json.gz
Resolving data.gharchive.org (data.gharchive.org)... 172.67.168.206, 104.21.46.175
Connecting to data.gharchive.org (data.gharchive.org)|172.67.168.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106793608 (102M) [application/gzip]
Saving to: ‘2021-09-01-15.json.gz.1’

2021-09-01-15.json. 100%[===================>] 101.85M  40.8MB/s    in 2.5s    

2021-09-11 16:30:31 (40.8 MB/s) - ‘2021-09-01-15.json.gz.1’ saved [106793608/106793608]



>Tip: You can edit the yyyy-mm-dd-hh format in `wget` command, depending upon which timeline you're interested in exploring

> Note: This is gzip json compressed file

## Built in Operators and Functions

1. **Dot(`.`)**
2. **pipe `|`**
3. **array operator `[]`**
4. **Output a CSV: `@csv`**
5. **Array and Object Construction: `[]` and `{}`**
6. **Filter `select()`**
7. **Apply a filter or function to an array `map()`** 
8. **Unique**
9. **length**
10. **group_by**

>Note: There is lot more in the [jq manual](https://stedolan.github.io/jq/manual/#Builtinoperatorsandfunctions) for you to look at.

Now again, switch back to your terminal to explore the raw data with jq

Reading a gzipped JSON file (printing only one record) and invoking jq on top of it

**1. Dot(`.`)**

jq . leaves the output unmodified. This is the basic operator and also useful when you're looking to prettify your json to be able to bring it in the readable format

In [ ]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 1 | jq .

{
  "id": "17808292672",
  "type": "PushEvent",
  "actor": {
    "id": 10745044,
    "login": "moises-santillan",
    "display_login": "moises-santillan",
    "gravatar_id": "",
    "url": "https://api.github.com/users/moises-santillan",
    "avatar_url": "https://avatars.githubusercontent.com/u/10745044?"
  },
  "repo": {
    "id": 271635192,
    "name": "moises-santillan/covid-19-mexico",
    "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
  },
  "payload": {
    "push_id": 7844628061,
    "size": 1,
    "distinct_size": 1,
    "ref": "refs/heads/master",
    "head": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
    "before": "cf3de81ab4584c0dcdbbee3e42bbeafc2d4b0c94",
    "commits": [
      {
        "sha": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
        "author": {
          "name": "Moisés Santillán",
          "email": "1b8f84bbb29551654b9a5d0b48cb0e5ec103b514@cinvestav.mx"
        },
        "message": "$",
        "distinct": true,
        "url": "https

You can use dot(.) operator to access one of the keys in json

In [33]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 1| jq .id

"17808292672"


Also, chaining the key with its nested key allows to access nested object

In [35]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 1| jq .repo.name

"moises-santillan/covid-19-mexico"


**2. pipe operator `|`** : You can use pipe and several operators together in jq for any complex transformations in your data

In [74]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 5| jq .type

"PushEvent"
"PullRequestEvent"
"DeleteEvent"
"WatchEvent"
"PushEvent"


**3. array operator `[]`** : This operator is particularly useful in cases when you want to represent a list of github events (JSON) in this case.

Following is the method you can use to create a list of github events in JSON

In [39]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 1 | jq '[.]'  

[
  {
    "id": "17808292672",
    "type": "PushEvent",
    "actor": {
      "id": 10745044,
      "login": "moises-santillan",
      "display_login": "moises-santillan",
      "gravatar_id": "",
      "url": "https://api.github.com/users/moises-santillan",
      "avatar_url": "https://avatars.githubusercontent.com/u/10745044?"
    },
    "repo": {
      "id": 271635192,
      "name": "moises-santillan/covid-19-mexico",
      "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
    },
    "payload": {
      "push_id": 7844628061,
      "size": 1,
      "distinct_size": 1,
      "ref": "refs/heads/master",
      "head": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
      "before": "cf3de81ab4584c0dcdbbee3e42bbeafc2d4b0c94",
      "commits": [
        {
          "sha": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
          "author": {
            "name": "Moisés Santillán",
            "email": "1b8f84bbb29551654b9a5d0b48cb0e5ec103b514@cinvestav.mx"
          },
         

Or you can also create arrays by just selecting few keys 

In [59]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 2 |  jq '[.id, .type, .repo]' 

[
  "17808292672",
  "PushEvent",
  {
    "id": 271635192,
    "name": "moises-santillan/covid-19-mexico",
    "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
  }
]
[
  "17808292724",
  "PullRequestEvent",
  {
    "id": 279965573,
    "name": "konveyor/forklift-ui",
    "url": "https://api.github.com/repos/konveyor/forklift-ui"
  }
]


In this case, above record have no keys, this becomes pretty handy when you want to map JSON to a csv file

**4. Output a CSV: `@csv`**

Before you output to CSV, make sure all the fields have been flattened, otherwise it would throw an error, like the one we see below

In [61]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 2 |  jq '[.id, .type, .repo]| @csv' 

jq: error (at <stdin>:1): object ({"id":27163...) is not valid in a csv row
jq: error (at <stdin>:2): object ({"id":27996...) is not valid in a csv row


In [64]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 3 | jq '[.id, .type, .repo.id,.repo.name,.repo.url]| @csv' 

"\"17808292672\",\"PushEvent\",271635192,\"moises-santillan/covid-19-mexico\",\"https://api.github.com/repos/moises-santillan/covid-19-mexico\""
"\"17808292724\",\"PullRequestEvent\",279965573,\"konveyor/forklift-ui\",\"https://api.github.com/repos/konveyor/forklift-ui\""
"\"17808292729\",\"DeleteEvent\",150458335,\"openvinotoolkit/model_server\",\"https://api.github.com/repos/openvinotoolkit/model_server\""


If you would like to truncate the double quotes in csv file, use the following

In [65]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 3 | jq '[.id, .type, .repo.id,.repo.name,.repo.url]| @csv' | tr -d '" '

\17808292672\,\PushEvent\,271635192,\moises-santillan/covid-19-mexico\,\https://api.github.com/repos/moises-santillan/covid-19-mexico\
\17808292724\,\PullRequestEvent\,279965573,\konveyor/forklift-ui\,\https://api.github.com/repos/konveyor/forklift-ui\
\17808292729\,\DeleteEvent\,150458335,\openvinotoolkit/model_server\,\https://api.github.com/repos/openvinotoolkit/model_server\


**5. Array and Object Construction**: `[]` and `{}`**

Let's say you want to use only few of the objects in json, there's a way to select few objects and create a new JSON. Following are the two methods:

In [72]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 2 |  jq '[.id, .type, .repo]' 

[
  "17808292672",
  "PushEvent",
  {
    "id": 271635192,
    "name": "moises-santillan/covid-19-mexico",
    "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
  }
]
[
  "17808292724",
  "PullRequestEvent",
  {
    "id": 279965573,
    "name": "konveyor/forklift-ui",
    "url": "https://api.github.com/repos/konveyor/forklift-ui"
  }
]


In [70]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 2 |  jq '{id:.id, type: .type, repo: .repo}' 

{
  "id": "17808292672",
  "type": "PushEvent",
  "repo": {
    "id": 271635192,
    "name": "moises-santillan/covid-19-mexico",
    "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
  }
}
{
  "id": "17808292724",
  "type": "PullRequestEvent",
  "repo": {
    "id": 279965573,
    "name": "konveyor/forklift-ui",
    "url": "https://api.github.com/repos/konveyor/forklift-ui"
  }
}


**6. `select()`**: select can be used for querying the JSON 

In [107]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 1 | jq  'select(.type=="PushEvent" and .actor.login=="moises-santillan")'

{
  "id": "17808292672",
  "type": "PushEvent",
  "actor": {
    "id": 10745044,
    "login": "moises-santillan",
    "display_login": "moises-santillan",
    "gravatar_id": "",
    "url": "https://api.github.com/users/moises-santillan",
    "avatar_url": "https://avatars.githubusercontent.com/u/10745044?"
  },
  "repo": {
    "id": 271635192,
    "name": "moises-santillan/covid-19-mexico",
    "url": "https://api.github.com/repos/moises-santillan/covid-19-mexico"
  },
  "payload": {
    "push_id": 7844628061,
    "size": 1,
    "distinct_size": 1,
    "ref": "refs/heads/master",
    "head": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
    "before": "cf3de81ab4584c0dcdbbee3e42bbeafc2d4b0c94",
    "commits": [
      {
        "sha": "e0dbca5c7f20d224377e6b25cbe1b7d165ce75f4",
        "author": {
          "name": "Moisés Santillán",
          "email": "1b8f84bbb29551654b9a5d0b48cb0e5ec103b514@cinvestav.mx"
        },
        "message": "$",
        "distinct": true,
        "url": "https

**7. Apply a filter or function to an array `map()`** : To collapse lists to a single top level element

In [54]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 5| jq -s 'map(.type)'

[
  "PushEvent",
  "PullRequestEvent",
  "DeleteEvent",
  "WatchEvent",
  "PushEvent"
]


**8. `unique`**

In [53]:
! gzip -q -d -c 2021-09-01-15.json.gz |  jq -s 'map(.type) | unique'

[
  "CommitCommentEvent",
  "CreateEvent",
  "DeleteEvent",
  "ForkEvent",
  "GollumEvent",
  "IssueCommentEvent",
  "IssuesEvent",
  "MemberEvent",
  "PublicEvent",
  "PullRequestEvent",
  "PullRequestReviewCommentEvent",
  "PullRequestReviewEvent",
  "PushEvent",
  "ReleaseEvent",
  "WatchEvent"
]


**9. `length`**

Here is what length means (taken from docs):

- The length of a string is the number of Unicode codepoints it contains (which will be the same as its JSON-encoded length in bytes if it's pure ASCII).

- The length of an array is the number of elements.

- The length of an object is the number of key-value pairs.

- The length of null is zero.

we have flattened the type list to top level elements and so it would return the number of elements, since it's an array

In [15]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 3 | jq -s 'map(.type)'

[
  "PushEvent",
  "PullRequestEvent",
  "DeleteEvent"
]


In [17]:
!gzip -q -d -c 2021-09-01-15.json.gz | head -n 3 | jq -s 'map(.type)| length'

3


**10. `group_by`**: chunk the results into grouped lists

In [3]:
! gzip -q -d -c 2021-09-01-15.json.gz | jq -s  'group_by (.type)[] | {type: .[0].type, count: length}'

{
  "type": "CommitCommentEvent",
  "count": 790
}
{
  "type": "CreateEvent",
  "count": 24577
}
{
  "type": "DeleteEvent",
  "count": 7878
}
{
  "type": "ForkEvent",
  "count": 2803
}
{
  "type": "GollumEvent",
  "count": 411
}
{
  "type": "IssueCommentEvent",
  "count": 13276
}
{
  "type": "IssuesEvent",
  "count": 4377
}
{
  "type": "MemberEvent",
  "count": 469
}
{
  "type": "PublicEvent",
  "count": 587
}
{
  "type": "PullRequestEvent",
  "count": 18561
}
{
  "type": "PullRequestReviewCommentEvent",
  "count": 3720
}
{
  "type": "PullRequestReviewEvent",
  "count": 5957
}
{
  "type": "PushEvent",
  "count": 76918
}
{
  "type": "ReleaseEvent",
  "count": 1012
}
{
  "type": "WatchEvent",
  "count": 7398
}


Now that we know enough functions and operators, we can leverage that to write a  statement which would aggregate **GH archive** logs on fields on interest. Let's see how we can do that. We would also look at how to add and delete fields in the aggregated JSON

**Aggregating the GH Archive logs**

**Adding** date_utc and date_hr two arguments in the JSON below and **deleting** actor_login

In [85]:
! gzip -q -d -c 2021-09-01-15.json.gz | head -n 3| jq -c --arg date_utc "$(date +'%d-%m-%Y')" --arg date_hr "$(date)" '. + {date_utc: $date_utc, date_hr : $date_hr}' |jq -s 'group_by (.date_utc, .date_hr,.actor.login,.repo.name,.type)[] | {"date_utc" : .[0].date_utc, "date_hr" : .[0].date_hr, "actor_login":.[0].actor.login,"repo_name":.[0].repo.name,"type": .[0].type, count: length}| del(.actor_login)' 

{
  "date_utc": "13-09-2021",
  "date_hr": "Mon Sep 13 17:20:46 IST 2021",
  "actor_login": "dtrawins",
  "repo_name": "openvinotoolkit/model_server",
  "type": "DeleteEvent",
  "count": 1
}
{
  "date_utc": "13-09-2021",
  "date_hr": "Mon Sep 13 17:20:46 IST 2021",
  "actor_login": "moises-santillan",
  "repo_name": "moises-santillan/covid-19-mexico",
  "type": "PushEvent",
  "count": 1
}
{
  "date_utc": "13-09-2021",
  "date_hr": "Mon Sep 13 17:20:46 IST 2021",
  "actor_login": "seanforyou23",
  "repo_name": "konveyor/forklift-ui",
  "type": "PullRequestEvent",
  "count": 1
}


## Shell-Script

Now, here is the example to the [shell-script](https://gist.github.com/rajkstats/41982bdd24bc68516bb53eab3a5e31e9) which was our end goal which would aggregate data on hourly granularity. Output is being written in csv format and removing double quotes. All the hourly files need to be downloaded in path being provided which would pick the files with `.gz` extension only. There is a room for lot of improvement in shell-script. A lot more structure can be provided here. Rather, treat this as a reference to get you started with `jq`.

**Snapshot of output of shell-script**

![shell script](agg-data-with-jq/shell-script.png)

## Next Steps
- Ingest the hourly aggregates in a database of your choice 
- Write an airflow DAG or CRON job to monitor the ETL pipeline

## Inspired From (Further Resources) 
- [jq Docs](https://stedolan.github.io/jq/)
- [Gh Archive](https://www.gharchive.org/)
- [JSON and jq](https://programminghistorian.org/en/lessons/json-and-jq#installation-on-os-x)
